In [6]:
from owslib.wms import WebMapService
wms = WebMapService("http://wms:8000/wms?", version="1.3.0")
wms.identification.title

"Rob's test OWS config"

In [7]:
list(wms.contents)

['ls8']

In [8]:
wms['ls8'].boundingBoxWGS84

(148.51023012109388,
 -35.712021629393874,
 151.19225656748156,
 -33.50349020610769)

In [9]:
wms['ls8'].styles

{'simple_rgb': {'title': 'Simple RGB'},
 'cloud_masked_rgb': {'title': 'Simple RGB with cloud masking'},
 'cloud_and_shadow_masked_rgb': {'title': 'Simple RGB with cloud and cloud shadow masking'},
 'extended_rgb': {'title': 'Extended RGB'},
 'wideband': {'title': 'Wideband false-colour'},
 'infra_red': {'title': 'False colour multi-band infra-red'},
 'coastal_aerosol': {'title': 'Spectral band 1 - Coastal aerosol'},
 'blue': {'title': 'Spectral band 2 - Blue'},
 'green': {'title': 'Spectral band 3 - Green'},
 'red': {'title': 'Spectral band 4 - Red'},
 'nir': {'title': 'Spectral band 5 - Near infra-red'},
 'swir1': {'title': 'Spectral band 6 - Short wave infra-red 1'},
 'swir2': {'title': 'Spectral band 7 - Short wave infra-red 2'},
 'ndvi': {'title': 'NDVI'},
 'ndvi_cloudmask': {'title': 'NDVI with cloud masking'},
 'ndwi': {'title': 'NDWI'},
 'ndwi_cloudmask': {'title': 'NDWI with cloud and cloud-shadow masking'},
 'ndbi': {'title': 'NDBI'},
 'cloud_mask': {'title': 'Cloud Mask'},
 

In [10]:
wms['ls8'].crsOptions

['EPSG:4326', 'EPSG:3577', 'EPSG:3857']

In [11]:
[op.name for op in wms.operations]

['GetCapabilities', 'GetMap', 'GetFeatureInfo']

In [12]:
wms.getOperationByName('GetMap').methods

[{'type': 'Get', 'url': 'http://wms:8000/wms?'}]

In [13]:
wms.getOperationByName('GetMap').formatOptions

['image/png']

In [14]:
# Set up centre of area to analyse, and a buffer in metres around this centrepoint
x, y, buffer_m = 16598849.27, -4204725.05, 10000
bbox = ((x - buffer_m),(y - buffer_m),(x + buffer_m),(y + buffer_m))

In [15]:
print(bbox)

(16588849.27, -4214725.05, 16608849.27, -4194725.05)


In [16]:
img = wms.getmap(layers=['ls8'],
                 styles=['simple_rgb'],
                 srs='EPSG:3857',
                 bbox=bbox,
                 size=(256,256),
                 format='image/png')
                 

In [17]:
out = open('test.png', 'wb')
out.write(img.read())
out.close()

In [18]:
img.geturl()

'http://wms:8000/wms?service=WMS&version=1.3.0&request=GetMap&layers=ls8&styles=simple_rgb&width=256&height=256&crs=EPSG%3A3857&bbox=16588849.27%2C-4214725.05%2C16608849.27%2C-4194725.05&format=image%2Fpng&transparent=FALSE&bgcolor=0xFFFFFF&exceptions=XML'

In [19]:

import ipywidgets as widgets

In [20]:
widgets.Image(
    value=img.read(),
    format='png',
    width=256,
    height=256,
)


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x00\x00\x00\x01\x00\x08\x02\x00\x00\x00\xd3\x10?…

In [21]:
from ipyleaflet import (
    Map, basemaps, basemap_to_tiles,
    WMSLayer, LayersControl
)

m = Map(center=(-35.3, 149.1),
        crs="EPSG3857",
        zoom=5)

#nasa_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-03-30")
#m.add_layer(nasa_layer)
# Use explicit IP address when in Docker as ipyleaflet fails to find the server by its name via Docker-compose name lookup
wms = WMSLayer(
    url="http://172.18.0.2:8000/?",
    layers="frac_cover",
    format="image/png",
    crs="EPSG3857",
    styles="tot_cover",
    name="test"
)
m.add_layer(wms)

m.add_control(LayersControl())

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [22]:
!curl -v -L --output test.png "http://wms:8000/wms?request=GetMap&format=image%2Fpng&version=1.3.0&crs=EPSG%3A3857&layers=ls8&bbox=149.05%2C-35.25%2C149.17%2C-35.35&width=256&height=256"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 172.17.0.3...
* TCP_NODELAY set
* Connected to wms (172.17.0.3) port 8000 (#0)
> GET /wms?request=GetMap&format=image%2Fpng&version=1.3.0&crs=EPSG%3A3857&layers=ls8&bbox=149.05%2C-35.25%2C149.17%2C-35.35&width=256&height=256 HTTP/1.1
> Host: wms:8000
> User-Agent: curl/7.58.0
> Accept: */*
> 
  0     0    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0< HTTP/1.1 200 OK
< Server: gunicorn/19.9.0
< Date: Mon, 16 Sep 2019 23:24:04 GMT
< Connection: close
< Access-Control-Allow-Origin: *
< Content-Type: image/png
< Content-Length: 155
< X-REQUEST-ID: 779b3a8e-f1b3-4cb9-b509-e1e8e975ed83
< 
  0   155    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0{ [155 bytes data]
100   155  100   155    0     0     22      0

In [12]:
!curl -v -L "http://wms:8000/?service=WMS&request=GetCapabilities"

*   Trying 172.18.0.2...
* TCP_NODELAY set
* Connected to wms (172.18.0.2) port 8000 (#0)
> GET /?service=WMS&request=GetCapabilities HTTP/1.1
> Host: wms:8000
> User-Agent: curl/7.58.0
> Accept: */*
> 
< HTTP/1.1 200 OK
< Server: gunicorn/19.9.0
< Date: Thu, 29 Aug 2019 05:02:56 GMT
< Connection: close
< Access-Control-Allow-Origin: *
< Content-Type: application/xml
< Cache-Control: no-cache,max-age=0
< Content-Length: 3815
< X-REQUEST-ID: e0d98911-2725-40ff-b41a-9ec8f14a387a
< 
<?xml version='1.0' encoding="UTF-8"?>
<WMS_Capabilities version="1.3.0"
xmlns="http://www.opengis.net/wms"
xmlns:xlink="http://www.w3.org/1999/xlink"
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
xsi:schemaLocation="http://www.opengis.net/wms
http://schemas.opengis.net/wms/1.3.0/capabilities_1_3_0.xsd">
<Service>
    <Name>WMS</Name>
    <Title>WMS server for Australian Landsat Datacube</Title>
    
    <Abstract>
        Historic Landsat imagery for Australia.
    </Abstract>
    
    
    <KeywordLi

In [66]:
!curl -h

Usage: curl [options...] <url>
     --abstract-unix-socket <path> Connect via abstract Unix domain socket
     --anyauth       Pick any authentication method
 -a, --append        Append to target file when uploading
     --basic         Use HTTP Basic Authentication
     --cacert <file> CA certificate to verify peer against
     --capath <dir>  CA directory to verify peer against
 -E, --cert <certificate[:password]> Client certificate file and password
     --cert-status   Verify the status of the server certificate
     --cert-type <type> Certificate file type (DER/PEM/ENG)
     --ciphers <list of ciphers> SSL ciphers to use
     --compressed    Request compressed response
     --compressed-ssh Enable SSH compression
 -K, --config <file> Read config from a file
     --connect-timeout <seconds> Maximum time allowed for connection
     --connect-to <HOST1:PORT1:HOST2:PORT2> Connect to host
 -C, --continue-at <offset> Resumed transfer offset
 -b, --cookie <data> Send cookies from string/